In [12]:
respondfile('20201218')

데이터 파싱 중:   0%|                                                                           | 0/92 [00:00<?, ?it/s]

품목 불러오기 완료


대응파일에 값 입력중: 100%|████████████████████████████████████████████████████████████████████| 92/92 [00:00<?, ?it/s]


데이터 파싱 완료
대응파일에 값 입력 완료


파일 만드는중: 100%|███████████████████████████████████████████████████████████████████████████| 92/92 [00:00<?, ?it/s]


파일 만들기 완료


In [2]:
def respondfile(s_date):
    xlsxload()
    fromparsing(s_date)
    xlsxpriceinput()
    xlsxtoxlsx(s_date)

진행시키는 파일

In [3]:
import requests
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm,trange

In [4]:
bslist = []#야채
bglist = []#등급
bwlist = []#규격
bplist = []#야채단가
test = []
testba = []
testbad = []
testgood = []

In [5]:
def priceload(s_date, s_pummok):
    url = 'http://www.garak.co.kr/publicdata/dataOpen.do?'
    id = '2860'
    pw = '!!Huko980412'
    dataid = 'data36'
    option = '&pagesize=10&pageidx=1&portal.templet=false'
    #s_date = '' #검색일
    s_date_p = '20181201' #검색일전일
    s_date_p7 = '20181126' #검색일7일전일
    s_pos_gubun = '1' #시장구분(가락시장 : 1, 강서시장 : 3)
    s_pum_nm = '2' #분류구분(청과 : 2, 수산 : 3)
    #s_pummok = '' #검색품목
    
    urlcompletion = url + 'id=' + id + '&passwd=' + pw + '&dataid=' + dataid + option + '&s_date=' + s_date + '&s_date_p=' + s_date_p + '&s_date_p7=' + s_date_p7 + '&p_pos_gubun=' + s_pos_gubun + '&s_pum_nm=' + s_pum_nm + '&s_pummok=' + s_pummok 
    return urlcompletion

상품명과 날짜를 넣어주면 자동으로 그에 맞게 데이터 검색 주소를 반환

In [6]:
def fromparsing(s_date):
    for i in trange(0, len(bslist), desc = '데이터 파싱 중'):
        if(bglist[i] != ''):
            data = BeautifulSoup(requests.get(priceload(s_date, bslist[i])).content).find_all('list')
            for item in data:
                if((item.pum_nm.string.replace(' ', '') == bslist[i].replace(' ', '')) and (item.g_name.string.replace(' ', '') == bglist[i]) and (item.u_name.string.replace(' ', '').lower() == bwlist[i])):
                    bplist.append(item.av_p.string.replace(' ', ''))
                    break
        else: bplist.append('')
    print('데이터 파싱 완료')

데이터 긁어오는 함수

In [7]:
def xlsxload():
    wb = openpyxl.load_workbook('C:/Users/Huko/garakparsing/야채단가 대응파일.xlsx')
    ws = wb.active


    for i in trange(1, ws.max_row, desc = '품목 불러오는 중'):
        if('End' == ws.cell(row = 3 + i, column = 1).value): break
        item = ws.cell(row = 3 + i, column = 2).value
        grade = ws.cell(row = 3 + i, column = 3).value
        standard = ws.cell(row = 3 + i, column = 4).value
        
        bslist.append(item)
        if(grade == None):
            bglist.append('')
            bwlist.append('')
        else:
            bglist.append(grade)
            bwlist.append(standard)
    print('품목 불러오기 완료')

대응파일에서 품목 리스트를 긁어오는 함수

In [8]:
def xlsxtoxlsx(s_date):
    wb = openpyxl.load_workbook('C:/Users/Huko/garakparsing/기본파일(12월 4일 기준).xlsx')
    ws = wb.active
    for i in trange(0, len(bplist), desc = '파일 만드는중'):
        ws.cell(row = 4 + i, column = 4).value = bplist[i]
    
    wb.save(s_date[0:4] + '년 ' + s_date[4:6] + '월 ' + s_date[6:8] + '일 ' + '야채단가' + '.xlsx')
    print('파일 만들기 완료')

야채단가를 넣고 파일을 만드는 함수

In [9]:
def xlsxpriceinput():
    wb = openpyxl.load_workbook('C:/Users/Huko/garakparsing/야채단가 대응파일.xlsx')
    ws = wb.active
    for i in trange(0, len(bplist), desc = '대응파일에 값 입력중'):
        ws.cell(row = 4 + i, column = 5).value = bplist[i]
    
    wb.save('C:/Users/Huko/garakparsing/야채단가 대응파일.xlsx')
    print('대응파일에 값 입력 완료')

긁어온 데이터를 대응파일에 붙여주는 함수

In [10]:
def testmodule(testtt):
    print(priceload('20201204', bslist[testtt]))

    data = BeautifulSoup(requests.get(priceload('20201204', bslist[testtt])).content).find_all('list')
    for item in data:
        if((item.pum_nm.string.replace(' ', '') == bslist[testtt].replace(' ', '')) and (item.g_name.string.replace(' ', '') == bglist[testtt]) and (item.u_name.string.replace(' ', '').lower() == bwlist[testtt])):
            print('발동')
            print(item.g_name.string.replace(' ', ''))
        if(item.pum_nm.string.replace(' ', '') == bslist[testtt]): print('이름 문제 없음' + "/" +'')
        if(item.g_name.string.replace(' ', '') == bglist[testtt]): print('급은 문제 없음' + "/" + '')
        if(item.u_name.string.replace(' ', '').lower() == bwlist[testtt]): print('단위 문제 없음')
        print(item.pum_nm.string.replace(' ', '') + '/' + item.g_name.string.replace(' ', '') + '/' + item.u_name.string.replace(' ', '').lower())

테스트를 위해 데이터를 정상적으로 긁어오는지 체크를 위한 함수

In [11]:
def printlist():
    for i in range(0, len(bslist)):
        print(str(i + 1) + " : " + bslist[i] + '/' + bglist[i] + '/' + bwlist[i] + '/' + bplist[i])

데이터 검토를 위해 긁어온 리스트들의 값을 보기좋게 불러와주는 함수